In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

In [2]:
from SCSCtrl import TTLServo
from jetbot import Robot
import time

Succeeded to open the port
Succeeded to change the baudrate


In [4]:
# camera setting
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [5]:
idx_servo_hori = 1
idx_servo_vert1 = 2
idx_servo_vert2 = 3
idx_servo_grip = 4
idx_servo_camera = 5
speed_motor = 150

In [6]:
dir = -1

In [7]:
degree_camera = 0
degree_hori = 0
degree_vert1 = 0
degree_vert2 = 0
degree_grip = 0

In [8]:
degree_motor = 5

In [9]:
degree_slider = widgets.IntSlider(description='degree', value = degree_motor, min=0, max=30, step=1, orientation='horizontal', continuous_update=False)

In [10]:
def update_degree(change):
    global degree_motor
    degree_motor = change.new
    print(f'Updated value: {degree_motor}')  # 변수 값 업데이트 시 출력

degree_slider.observe(update_degree, names='value')

In [11]:
print(degree_motor)

5


In [12]:
def check_in_range(degree):
    if degree > 150:
        return 150
    if degree < -150:
        return -150
    return degree

In [13]:
def control_horizontal(degree):
    global idx_servo_hori, speed_motor
    global degree_hori, dir
    degree_hori = check_in_range(degree + degree_hori)
    TTLServo.servoAngleCtrl(idx_servo_hori, degree_hori, dir, speed_motor)

def control_vertical1(degree):
    global idx_servo_vert1, speed_motor
    global degree_vert1, dir
    degree_vert1 = check_in_range(degree + degree_vert1)
    TTLServo.servoAngleCtrl(idx_servo_vert1, degree_vert1, dir, speed_motor)
    
def control_vertical2(degree):
    global idx_servo_vert2, speed_motor
    global degree_vert2, dir
    degree_vert2 = check_in_range(degree + degree_vert2)
    TTLServo.servoAngleCtrl(idx_servo_vert2, degree_vert2, dir, speed_motor)
    
def control_grip(degree):
    global idx_servo_grip, speed_motor
    global degree_grip, dir
    degree_grip = check_in_range(degree + degree_grip)
    TTLServo.servoAngleCtrl(idx_servo_grip, degree_grip, dir, speed_motor)

def control_camera(degree):
    global idx_servo_camera, speed_motor
    global degree_camera, dir
    degree_camera = check_in_range(degree + degree_camera)
    print(degree_camera)
    TTLServo.servoAngleCtrl(idx_servo_camera, degree_camera, dir, speed_motor)

In [14]:
# @type is whether staying grip or release
def init(type=True):
    global idx_servo_hori, idx_servo_vert1, idx_servo_vert2, idx_servo_grip, idx_servo_camera, speed_motor
    global degree_hori, degree_vert1, degree_vert2, degree_grip, degree_camera, dir
    degree_hori = -8
    degree_vert1 = 0
    degree_vert2 = 0
    degree_grip = -8
    degree_camera = 0
    TTLServo.servoAngleCtrl(idx_servo_hori, degree_hori, dir, speed_motor)
    TTLServo.servoAngleCtrl(idx_servo_vert1, degree_vert1, dir, speed_motor)
    TTLServo.servoAngleCtrl(idx_servo_vert2, degree_vert2, dir, speed_motor)
    if type:
        TTLServo.servoAngleCtrl(idx_servo_grip, degree_grip, dir, speed_motor)
    TTLServo.servoAngleCtrl(idx_servo_camera, degree_camera, dir, speed_motor)

In [15]:
init()

In [16]:
PT_layout = widgets.Layout(width='100px', height='80px', align_self='center')

In [17]:
cameraUp_button = widgets.Button(description='LookUp', button_style='success', layout=PT_layout)
cameraDown_button = widgets.Button(description='LookDown', button_style='success', layout=PT_layout)
cameraUp_button.on_click(lambda x: control_camera(degree_motor))
cameraDown_button.on_click(lambda x: control_camera(-1 * degree_motor))

bodyLeft_button = widgets.Button(description='LookLeft', button_style='success', layout=PT_layout)
bodyRight_button = widgets.Button(description='LookRight', button_style='success', layout=PT_layout)
bodyLeft_button.on_click(lambda x: control_horizontal(degree_motor))
bodyRight_button.on_click(lambda x: control_horizontal(-1 * degree_motor))

grap_button = widgets.Button(description='Grap', button_style='success', layout=PT_layout)
Release_button = widgets.Button(description='Release', button_style='success', layout=PT_layout)
grap_button.on_click(lambda x: control_grip(degree_motor))
Release_button.on_click(lambda x: control_grip(-1 * degree_motor))

vert1Up_button = widgets.Button(description='Vert1 Up', button_style='success', layout=PT_layout)
vert1Down_button = widgets.Button(description='Vert1 Down', button_style='success', layout=PT_layout)
vert1Up_button.on_click(lambda x: control_vertical1(degree_motor))
vert1Down_button.on_click(lambda x: control_vertical1(-1 * degree_motor))

vert2Up_button = widgets.Button(description='Vert2 Up', button_style='success', layout=PT_layout)
vert2Down_button = widgets.Button(description='Vert2 Down', button_style='success', layout=PT_layout)
vert2Up_button.on_click(lambda x: control_vertical2(-1 * degree_motor))
vert2Down_button.on_click(lambda x: control_vertical2(degree_motor))

topBtn_box = widgets.HBox([cameraUp_button, bodyLeft_button, grap_button, vert1Up_button, vert2Up_button], layout=widgets.Layout(align_self='center'))
bottomBtn_box = widgets.HBox([cameraDown_button, bodyRight_button, Release_button, vert1Down_button, vert2Down_button], layout=widgets.Layout(align_self='center'))
surPT_box = widgets.VBox([topBtn_box, bottomBtn_box, degree_slider])

display(surPT_box)

In [19]:
robot = Robot()

In [20]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_backward(change):
    robot.backward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

In [21]:
# define move buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

In [22]:
print(degree_camera)
print(degree_hori) 
print(degree_vert1)
print(degree_vert2)
print(degree_grip) 

0
-8
0
0
-8


# Training Widget

In [23]:
import os

good_dir = 'dataset/good'
left_dir = 'dataset/left'
right_dir = 'dataset/right'
top_dir = 'dataset/top'
down_dir = 'dataset/down'
far_dir = 'dataset/far'
near_dir = 'dataset/near'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(good_dir)
    os.makedirs(left_dir)
    os.makedirs(right_dir)
    os.makedirs(top_dir)
    os.makedirs(down_dir)
    os.makedirs(far_dir)
    os.makedirs(near_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

In [24]:
dir_list = [good_dir, left_dir, right_dir, top_dir, down_dir, far_dir, near_dir]

In [25]:
button_layout = widgets.Layout(width='128px', height='64px')

In [26]:
good_button = widgets.Button(description='add good', button_style='success', layout=button_layout)
left_button = widgets.Button(description='add left', button_style='warning', layout=button_layout)
right_button = widgets.Button(description='add right', button_style='warning', layout=button_layout)
top_button = widgets.Button(description='add top', button_style='warning', layout=button_layout)
down_button = widgets.Button(description='add down', button_style='warning', layout=button_layout)
far_button = widgets.Button(description='add far', button_style='warning', layout=button_layout)
near_button = widgets.Button(description='add near', button_style='warning', layout=button_layout)
button_list = [good_button, left_button, right_button, top_button, down_button, far_button, near_button]

In [27]:
good_count = widgets.IntText(layout=button_layout, value=len(os.listdir(good_dir)))
left_count = widgets.IntText(layout=button_layout, value=len(os.listdir(left_dir)))
right_count = widgets.IntText(layout=button_layout, value=len(os.listdir(right_dir)))
top_count = widgets.IntText(layout=button_layout, value=len(os.listdir(top_dir)))
down_count = widgets.IntText(layout=button_layout, value=len(os.listdir(down_dir)))
far_count = widgets.IntText(layout=button_layout, value=len(os.listdir(far_dir)))
near_count = widgets.IntText(layout=button_layout, value=len(os.listdir(near_dir)))
count_list = [good_count, left_count, right_count, top_count, down_count, far_count, near_count]

In [28]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_to_dir(idx):
    global dir_list, button_list, count_list
    save_snapshot(dir_list[idx])
    print(idx)
    count_list[idx].value = len(os.listdir(dir_list[idx]))
        
        
# def save_free():
#     global free_dir, free_count
#     save_snapshot(free_dir)
#     free_count.value = len(os.listdir(free_dir))

    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
    
# free_button.on_click(lambda x: save_free())
# blocked_button.on_click(lambda x: save_blocked())

In [29]:
button_list[0].on_click(lambda x: save_to_dir(0))
button_list[1].on_click(lambda x: save_to_dir(1))
button_list[2].on_click(lambda x: save_to_dir(2))
button_list[3].on_click(lambda x: save_to_dir(3))
button_list[4].on_click(lambda x: save_to_dir(4))
button_list[5].on_click(lambda x: save_to_dir(5))
button_list[6].on_click(lambda x: save_to_dir(6))

In [30]:
display(image)
display(widgets.HBox(button_list))
display(widgets.HBox(count_list))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [31]:
!zip -r -q dataset.zip dataset